In [1]:
import sympy as sp

In [2]:
t, v = sp.symbols('t v', real=True)
r = sp.Symbol('r', real=True, positive=True)
v_dot = sp.Symbol('\\dot{v}', real=True)
theta, phi = sp.symbols('\\theta \\phi', real=True)
r_dot, phi_dot, theta_dot = sp.symbols('\\dot{r} \\dot{\\phi} \\dot{\\theta}', real=True)
r_ddot, theta_ddot, phi_ddot = sp.symbols('\\ddot{r} \\ddot{\\theta} \\ddot{\\phi}', real=True)
M, c, e = sp.symbols('M c e', positive = True)

p_phi, r_0, v_0, theta_rv = sp.symbols('p_{\\phi} r_0 v_0 \\theta_rv', real=True)

U_e = sp.Function('U')(r)

r_t = sp.Function('r')(t)
theta_t = sp.Function('theta')(t)
phi_t = sp.Function('phi')(t)
v_t = sp.Function('v')(t)

dr_dt = r_t.diff(t)
dtheta_dt = theta_t.diff(t)
dphi_dt = phi_t.diff(t)

In [3]:
t2v = {
    r_t.diff(t).diff(t): r_ddot,
    r_t.diff(t): r_dot,
    r_t: r, 
    phi_t.diff(t).diff(t): phi_ddot,
    phi_t.diff(t): phi_dot,
    phi_t: phi,
    theta_t.diff(t).diff(t): theta_ddot,
    theta_t.diff(t): theta_dot,
    theta_t: theta,
    v_t.diff(t): v.diff(t),
    v_t: v,
}
t2v

{Derivative(r(t), (t, 2)): \ddot{r},
 Derivative(r(t), t): \dot{r},
 r(t): r,
 Derivative(phi(t), (t, 2)): \ddot{\phi},
 Derivative(phi(t), t): \dot{\phi},
 phi(t): \phi,
 Derivative(theta(t), (t, 2)): \ddot{\theta},
 Derivative(theta(t), t): \dot{\theta},
 theta(t): \theta,
 Derivative(v(t), t): 0,
 v(t): v}

In [4]:
v2t = {
    r_ddot: r_t.diff(t).diff(t),
    r_dot: r_t.diff(t),
    r: r_t,
    phi_ddot: phi_t.diff(t).diff(t),
    phi_dot: phi_t.diff(t),
    phi: phi_t,
    theta_ddot: theta_t.diff(t).diff(t),
    theta_dot: theta_t.diff(t),
    theta: theta_t,
    v_dot: v_t.diff(t),
    v: v_t,
}
v2t

{\ddot{r}: Derivative(r(t), (t, 2)),
 \dot{r}: Derivative(r(t), t),
 r: r(t),
 \ddot{\phi}: Derivative(phi(t), (t, 2)),
 \dot{\phi}: Derivative(phi(t), t),
 \phi: phi(t),
 \ddot{\theta}: Derivative(theta(t), (t, 2)),
 \dot{\theta}: Derivative(theta(t), t),
 \theta: theta(t),
 \dot{v}: Derivative(v(t), t),
 v: v(t)}

In [5]:
L = 1 / 2 * M * v ** 2 - U_e
L

0.5*M*v**2 - U(r)

## Developing $\ddot{r}$ equation
the Euler-Lagrange equation is: $\frac{\partial}{\partial t} (\frac{\partial L}{\partial \dot r})= \frac{\partial L}{\partial r}$


In [6]:
v_squared = r_dot ** 2 + r ** 2 * theta_dot ** 2 + r ** 2 * sp.sin(theta) ** 2 * phi_dot ** 2
f_gamma = (1 / sp.sqrt(1 - v ** 2 / c ** 2))

Notice that by differentiation by chain rule $\frac{\partial L}{\partial \dot r} = \frac{\partial L}{\partial v} \frac{\partial v}{\partial \dot{r}} = \frac{\partial L}{\partial v} \frac{\partial v}{\partial \dot{r}} $

In [7]:
dL_dv = L.diff(v)
dL_dv

1.0*M*v

In [8]:
dv_drdot = sp.sqrt(v_squared).diff(r_dot)
dv_drdot = dv_drdot.subs(sp.sqrt(v_squared), v)
dv_drdot

\dot{r}/v

In [9]:
dL_drdot = dL_dv * dv_drdot
dL_drdot = dL_drdot.simplify()
dL_drdot

1.0*M*\dot{r}

In [10]:
dL_drdot_t = dL_drdot.subs(v2t)
dL_drdot_t = dL_drdot_t.simplify()
dL_drdot_t

1.0*M*Derivative(r(t), t)

And here is the left side of the Euler-Lagrange equation:

In [11]:
d_dt_dL_drdot = dL_drdot_t.diff(t)
d_dt_dL_drdot

1.0*M*Derivative(r(t), (t, 2))

Now lets start with the right side of the Euler-Lagrange equation $\frac{\partial L}{\partial r}$,
Notice that by differentiation by chain rule:

$\frac{\partial L}{\partial r} = \frac{\partial L}{\partial v} \frac{\partial v}{\partial r} - \frac{\partial}{\partial r} U(r)$


In [12]:
dv_dr = sp.sqrt(v_squared).diff(r)
dv_dr = dv_dr.subs(sp.sqrt(v_squared), v)
dv_dr

(\dot{\phi}**2*r*sin(\theta)**2 + \dot{\theta}**2*r)/v

In [13]:
dL_dr = dL_dv * dv_dr - U_e.diff(r)
dL_dr = dL_dr.simplify()
dL_dr

1.0*M*r*(\dot{\phi}**2*sin(\theta)**2 + \dot{\theta}**2) - Derivative(U(r), r)

In [14]:
r_ddot_eq = d_dt_dL_drdot - dL_dr.subs(v2t)
r_ddot_eq = (r_ddot_eq / M).simplify()
r_ddot_eq.simplify()

(-1.0*M*(sin(theta(t))**2*Derivative(phi(t), t)**2 + Derivative(theta(t), t)**2)*r(t) + 1.0*M*Derivative(r(t), (t, 2)) + Derivative(U(r(t)), r(t)))/M

In [15]:
r_ddot_eq = r_ddot_eq.subs(t2v)
r_ddot_eq

(1.0*M*\ddot{r} - 1.0*M*r*(\dot{\phi}**2*sin(\theta)**2 + \dot{\theta}**2) + Derivative(U(r), r))/M

In [16]:
r_ddot_s_0 = sp.solve(r_ddot_eq, r_ddot)[0]
r_ddot_s_0

(M*r*(\dot{\phi}**2*sin(\theta)**2 + \dot{\theta}**2) - Derivative(U(r), r))/M

### Developing $\dot{\phi}$ equation
Lets find what $\dot{\phi}$ equals to, using the Euler-Lagrange equation for $\phi$:

Notice that:
$\frac{\partial L}{\partial \dot{\phi}} = \frac{\partial L}{\partial v} \frac{\partial v}{\partial \dot{\phi}}$

In [17]:
dv_dphidot = sp.sqrt(v_squared).diff(phi_dot)
dv_dphidot = dv_dphidot.subs(sp.sqrt(v_squared), v)
dv_dphidot

\dot{\phi}*r**2*sin(\theta)**2/v

In [18]:
dL_dphidot = dL_dv * dv_dphidot
dL_dphidot

1.0*M*\dot{\phi}*r**2*sin(\theta)**2

In [19]:
phi_dot_eq = sp.solve(dL_dphidot - p_phi, phi_dot)[0]
phi_dot_eq

p_{\phi}/(M*r**2*sin(\theta)**2)

In [20]:
dphi_dt_eq = phi_dot_eq.subs(v2t)
dphi_dt_eq.expand()

p_{\phi}/(M*r(t)**2*sin(theta(t))**2)

In [21]:
dphi_dtt_eq = dphi_dt_eq.diff(t).subs(t2v).subs(phi_dot_eq, phi_dot)
dphi_dtt_eq.expand()

-2*\dot{\phi}*\dot{\theta}*cos(\theta)/sin(\theta) - 2*\dot{\phi}*\dot{r}/r

From the Euler-Lagrange equation: $\frac{\partial}{\partial t} (\frac{\partial L}{\partial \dot{\phi}}) = 0 \to \gamma M r^2 \dot{\phi} = p_{\phi} \to \dot{\phi} = \frac{p_{\phi}}{\gamma M r^2}$

In [22]:
f_v_t = sp.sqrt(v_squared).subs(v2t)
dv_dt = f_v_t.diff(t).subs(f_v_t, v_t)
dv_dt

(r(t)**2*sin(theta(t))**2*Derivative(phi(t), t)*Derivative(phi(t), (t, 2)) + r(t)**2*sin(theta(t))*cos(theta(t))*Derivative(phi(t), t)**2*Derivative(theta(t), t) + r(t)**2*Derivative(theta(t), t)*Derivative(theta(t), (t, 2)) + r(t)*sin(theta(t))**2*Derivative(phi(t), t)**2*Derivative(r(t), t) + r(t)*Derivative(r(t), t)*Derivative(theta(t), t)**2 + Derivative(r(t), t)*Derivative(r(t), (t, 2)))/v(t)

In [23]:
dv_dt.subs(t2v)

(\ddot{\phi}*\dot{\phi}*r**2*sin(\theta)**2 + \ddot{\theta}*\dot{\theta}*r**2 + \ddot{r}*\dot{r} + \dot{\phi}**2*\dot{\theta}*r**2*sin(\theta)*cos(\theta) + \dot{\phi}**2*\dot{r}*r*sin(\theta)**2 + \dot{\theta}**2*\dot{r}*r)/v

In [24]:
phi_dot_eq_t = phi_dot_eq.subs(v2t)
phi_dot_eq_t

p_{\phi}/(M*r(t)**2*sin(theta(t))**2)

In [25]:
phi_dot_eq_0 = sp.Eq(phi_dot, phi_dot_eq.expand())
expr = phi_dot * r
phi_dot_eq_0 = sp.Eq(phi_dot_eq_0.lhs * expr, (phi_dot_eq_0.rhs * expr).expand())
expr = phi_dot * p_phi  / M / r / sp.sin(theta) ** 2
phi_dot_eq_0 = sp.Eq(phi_dot_eq_0.lhs - expr, (phi_dot_eq_0.rhs - expr).expand())
phi_dot_eq_0 = sp.Eq(-phi_dot_eq_0.lhs, -phi_dot_eq_0.rhs)
phi_dot_eq_0

Eq(-\dot{\phi}**2*r + \dot{\phi}*p_{\phi}/(M*r*sin(\theta)**2), 0)

In [26]:
r_ddot_s_1 = r_ddot_s_0.subs(phi_dot_eq_0.rhs, phi_dot_eq_0.lhs).simplify()
r_ddot_s_1

(M*r*(\dot{\phi}**2*sin(\theta)**2 + \dot{\theta}**2) - Derivative(U(r), r))/M

In [27]:
r_ddot_s_1

(M*r*(\dot{\phi}**2*sin(\theta)**2 + \dot{\theta}**2) - Derivative(U(r), r))/M

## Developing $\ddot{\theta}$ equation

In [28]:
theta_v = sp.sqrt(r_dot ** 2 + r ** 2 * theta_dot ** 2 + r ** 2 * sp.sin(theta) ** 2 * phi_dot ** 2)
theta_v

sqrt(\dot{\phi}**2*r**2*sin(\theta)**2 + \dot{\theta}**2*r**2 + \dot{r}**2)

$\frac{\partial L}{\partial \dot{\theta}} = \frac{\partial L}{\partial v} \frac{\partial v}{\partial \dot{\theta}} $

In [29]:
dv_dthetadot = theta_v.diff(theta_dot)
dv_dthetadot = dv_dthetadot.subs(theta_v, v)
dv_dthetadot

\dot{\theta}*r**2/v

In [30]:
dv_dtheta = theta_v.diff(theta)
dv_dtheta = dv_dtheta.subs(theta_v, v)
dv_dtheta

\dot{\phi}**2*r**2*sin(\theta)*cos(\theta)/v

In [31]:
dL_dthetadot = dL_dv * dv_dthetadot
dL_dthetadot

1.0*M*\dot{\theta}*r**2

In [32]:
dL_dtheta = dL_dv * dv_dtheta
dL_dtheta

1.0*M*\dot{\phi}**2*r**2*sin(\theta)*cos(\theta)

In [33]:
dL_dthetadot_t = dL_dthetadot.subs(v2t)
dL_dthetadot_t = dL_dthetadot_t.simplify()
d_dt_dL_dthetadot = dL_dthetadot_t.diff(t)
d_dt_dL_dthetadot

1.0*M*r(t)**2*Derivative(theta(t), (t, 2)) + 2.0*M*r(t)*Derivative(r(t), t)*Derivative(theta(t), t)

In [34]:
dL_dtheta_t = dL_dtheta.subs(v2t)
dL_dtheta_t = dL_dtheta_t.simplify()
dL_dtheta_t

0.5*M*r(t)**2*sin(2*theta(t))*Derivative(phi(t), t)**2

In [35]:
theta_ddot_eq_t = d_dt_dL_dthetadot - dL_dtheta_t
theta_ddot_eq_t = theta_ddot_eq_t.simplify()
theta_ddot_eq_t

M*(-0.5*r(t)*sin(2*theta(t))*Derivative(phi(t), t)**2 + 1.0*r(t)*Derivative(theta(t), (t, 2)) + 2.0*Derivative(r(t), t)*Derivative(theta(t), t))*r(t)

In [36]:
theta_ddot_eq = theta_ddot_eq_t.subs(t2v)
theta_ddot_eq = theta_ddot_eq.simplify()
theta_ddot_eq

M*r*(1.0*\ddot{\theta}*r - 0.5*\dot{\phi}**2*r*sin(2*\theta) + 2.0*\dot{\theta}*\dot{r})

In [37]:
theta_ddot_s_0 = sp.solve(theta_ddot_eq, theta_ddot)[0]
theta_ddot_s_0

0.5*\dot{\phi}**2*sin(2.0*\theta) - 2.0*\dot{\theta}*\dot{r}/r

In [38]:
theta_ddot_s_1 = sp.solve(theta_ddot_eq, theta_ddot)[0]
theta_ddot_s_1

0.5*\dot{\phi}**2*sin(2.0*\theta) - 2.0*\dot{\theta}*\dot{r}/r

In [39]:
r_ddot_s_1.expand()

\dot{\phi}**2*r*sin(\theta)**2 + \dot{\theta}**2*r - Derivative(U(r), r)/M

In [41]:
phi_dot_eq

p_{\phi}/(M*r**2*sin(\theta)**2)

In [40]:
r_ddot_s_1.subs(phi_dot, phi_dot_eq).expand()

\dot{\theta}**2*r - Derivative(U(r), r)/M + p_{\phi}**2/(M**2*r**3*sin(\theta)**2)

## In Summary the equations of motion:
- $\ddot r = \dot{\phi}^{2} r \sin^{2}{\left(\theta \right)} + \dot{\theta}^{2} r - \frac{\frac{d}{dr}U(r)}{M}$

- $\ddot \theta = \dot{\phi}^{2} \sin{\left(\theta \right)} \cos{\left(\theta \right)} - \frac{2 \dot{\theta} \dot{r}}{r}$

- $ \dot \phi = \frac{p_{\phi}}{M r^{2} \sin^{2}{\left(\theta \right)}}$